**тематическое моделирование**

Евгений Борисов borisov.e@solarl.ru

In [1]:
# http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

# разложение частотной матрицы [ слова x документы ]  
#
# получаем матрицу с описанием тем [ слова х темы ]   
# и матрицу вероятностей событий "тема описывает документ"  [ темы х документы ]
# 
# [ слова x документы ] = [ слова х темы ] * [ темы х документы ]
# 
# p(w|d) = p(w|t) * p(t|d)

In [2]:
import sys
import re
import gzip
import pandas as pd

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

In [4]:
pd.options.display.max_colwidth = 200  

In [5]:
n_features = 1000
n_components = 10

df = pd.read_pickle('../data/text/news.pkl.gz')
print('текстов:',len(df))
df.sample(4)

текстов: 3196


,text,tag
2846,"Телеканал «Пятница» сообщил, что арбитражный суд Москвы принял иск о защите деловой репутации канала. Поводом для него стало обсуждение в фейсбуке выпуска программы «Ревизорро», в котором ведущая ...",incident
2459,В Уфе пройдет большой концерт с участием известных кубызистов республики.\n\n\n\nВ Башгосфилармонии в рамках проекта «Бессмертное наследие моего народа» состоится концерт «Магический кубыз». В муз...,culture
2815,"Проект Google Earth Engine в режиме Timelapse позволяет проследить историю изменений, происходивших на поверхности Земли за последние 30 лет.\n\nРазработчики обработали более 5 млн фотографий со с...",tech
2874,"Корейский автопроизводитель планировал начать продажи нового Tivoli в России ещё в ноябре, но уже прошла половина декабря, а официальных новостей о новинке по-прежнему нет.\n\nВ российской пресс-с...",auto


In [6]:
data = df['text'].tolist()

In [8]:
# with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
#     stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

# from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords

stopwords = set(nltk_stopwords.words('russian') )
print('количество стоп-слов:',len(stopwords))

sorted(stopwords)

количество стоп-слов: 151


['а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [9]:
%%time 

df['text_clean'] = df['text'].str.lower() # приведение в lowercase

# замена символов-разделителей (-,_) на пробел
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'_', ' ', s))

# замена цифр
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' ', s))

# делим строки на слова
df['text_clean'] = df['text_clean'].apply(lambda t: [ w.strip() for w in t.split() if len(w.strip())>2 ] )

# удаление лишних слов
df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])

CPU times: user 572 ms, sys: 30.1 ms, total: 603 ms
Wall time: 604 ms


In [10]:
df[['text_clean']].sample(10)

,text_clean
320,"[вячеслав, малежик, сегодня, споет, вологду, русском, доме, вячеслав, малежик, вологодчине, первый, словам, певца, вологодский, край, произвел, сильное, впечатление]"
3137,"[традициях, финно, угорских, самодийских, народов, расскажет, новый, фильм, новости, online, регион, информ, москва, реализации, программы, улучшение, жилищных, условий, населения, ханты, мансийск..."
293,"[standard, poor, повысило, рейтинг, управления, мтс, агентство, standard, poor, повысило, рейтинг, корпоративного, управления, рку, компании, мобильные, телесистемы, российской, шкале, уровня, рку..."
2742,"[гознак, выпускающее, российские, деньги, зарегистрировало, патент, банкноты, прозрачными, участками, светящимися, волокнами, новые, технологии, должны, усилить, защиту, денег, подделок, такая, ин..."
2151,"[роналду, побед, реалом, лиге, чемпионов, сборной, португалии, евро, изначально, считался, фаворитом, голосования, подумал, смогу, четыре, раза, стать, обладателем, золотого, мяча, криштиану, рона..."
1643,"[ближайшее, время, произойти, сильное, извержение, вулкана, безымянный, сообщает, риа, новости, ссылкой, главу, камчатской, группы, реагирования, вулканические, извержения, kvert, института, вулка..."
455,"[кандидат, повышение, почему, премьер, министр, франции, ушел, отставку, декабря, рбк, президент, франции, франсуа, олланд, вторник, назначил, главой, правительства, министра, внутренних, дел, бер..."
2895,"[южнокорейский, автоконцерн, kia, выложил, открытый, доступ, тизер, своего, нового, спортивного, автомобиля, kia, новинку, называют, самым, динамичным, автомобилем, линейке, киа, судя, видеосюжету..."
2373,"[появилось, видео, момента, наезда, водителя, mercedes, двух, школьниц, западе, москвы, кадрах, видно, иномарка, сносит, детей, прямо, пешеходном, переходе, передаёт, звезда, ранее, сообщалось, ин..."
2221,"[внезапное, недомогание, водителя, рейсового, автобуса, привело, аварии, районе, дома, проспекту, солидарности, утром, декабря, водитель, автобуса, маршрута, прошел, медосмотр, здоров, однако, нес..."


In [11]:
# sorted(set([ w for t in  df['text_clean'] for w in t ]))

In [12]:
# собираем слова в строку
df['text_clean'] = df['text_clean'].apply(lambda t: ' '.join(t) )

In [13]:
%xdel stopwords

In [14]:
df[ df['text_clean'].str.len()<1 ]

,text,tag,text_clean


In [15]:
df.sample(10)

,text,tag,text_clean
1654,"Газета «Коммерсантъ» сообщила о том, что Россия якобы предложила фигуранту дела Немцова Темирлану Эскерханову компенсацию после того, как он пожаловался в ЕСПЧ.\n\nПо данным издания, речь идет о с...",politics,газета коммерсантъ сообщила россия якобы предложила фигуранту дела немцова темирлану эскерханову компенсацию пожаловался еспч данным издания речь идет сумме шесть тысяч евро журналисты также указы...
670,Клубные карты в фитнес-клуб World Class уже в продаже - скидка 40%\n\n29 ноября 2016 в 9:00\n\n*На правах рекламы\n\nПолноформатный семейный клуб World Class LITE Малиновка откроется в феврале 201...,reclama,клубные карты фитнес клуб world class продаже скидка ноября правах рекламы полноформатный семейный клуб world class lite малиновка откроется феврале адресу рафиева декабря клубные карты приобрести...
240,В ПОЛЬШЕ УСТАНОВИЛИ САМУЮ ВЫСОКУЮ В ЕВРОПЕ НОВОГОДНЮЮ ЕЛКУ 6 декабря в\nВаршаве зажглась самая высокая новогодняя елка в Европе. 72- метровая\nискусственная елка зажглась в центре польской столицы...,politics,польше установили самую высокую европе новогоднюю елку декабря варшаве зажглась самая высокая новогодняя елка европе метровая искусственная елка зажглась центре польской столицы зданием дворца нау...
2271,"Один из очевидцев массовой аварии на автомагистрали Кемерово – Ленинск-Кузнецкий выложил в Сеть видео последствий происшествия. Ролик появился в соцсети «ВКонтакте».\n\nКак видно на записи, разбит...",incident,очевидцев массовой аварии автомагистрали кемерово ленинск кузнецкий выложил сеть видео последствий происшествия ролик появился соцсети вконтакте видно записи разбитыми машинами занято несколько ме...
2263,"Взрыв прогремел на военной базе в Южной Корее. Более 20 человек пострадали. Об этом во вторник сообщает агентство Рёнхап.\n\nОтмечается, что всеми ранеными являются военнослужащие, многие из них п...",incident,взрыв прогремел военной базе южной корее человек пострадали вторник сообщает агентство рёнхап отмечается всеми ранеными являются военнослужащие многие получили ожоги госпитализированы причина взры...
2264,"СЕУЛ, 13 декабря. /Корр. ТАСС Станислав Варивода/. Двадцать три военнослужащих пострадали в результате взрыва, произошедшего на военной базе в районе южнокорейского города Ульсан. Как сообщает аге...",incident,сеул декабря корр тасс станислав варивода двадцать военнослужащих пострадали результате взрыва произошедшего военной базе районе южнокорейского города ульсан сообщает агентство yonhap трое находят...
686,"""Ставят под сомнение суверенитет Беларуси"". Мининформ — о задержаниях авторов российских СМИ\n\n9 декабря 2016 в 22:54\n\nTUT.BY\n\nМинистр информации Лилия Ананич прокомментировала задержания авт...",politics,ставят сомнение суверенитет беларуси мининформ задержаниях авторов российских сми декабря tut министр информации лилия ананич прокомментировала задержания авторов российских сайтов следственный ко...
2577,"Сегодня, 10:44\n\nЖители нашей планеты будут наблюдать последнее в уходящем году суперлуние. Космическое явление ожидается в ночь с 13 на 14 декабря в 4 часа 5 минут по Московскому часовому поясу....",science,сегодня жители нашей планеты будут наблюдать последнее уходящем году суперлуние космическое явление ожидается ночь декабря часа минут московскому часовому поясу небесное тело третье счету году вид...
1680,© Сергей Бобылев/ТАСС\n\nВ декабре 1991 года распался Советский Союз. 25 декабря первый президент СССР Михаил Горбачев сложил с себя полномочия главы государства.\n\nВ преддверии 25-летия этой дат...,politics,сергей бобылев тасс декабре года распался советский союз декабря первый президент ссср михаил горбачев сложил полномочия главы государства преддверии летия даты бывший советский лидер интервью тас...
2865,"Наиболее доступной по цене для россиян окажется комплектация SsangYong Tivoli Welcome, стоимость которой составляет 999 тысяч рублей. Данная версия включает подушки безопасности, ABS, кондиционер ...",auto,наиболее доступной цене россиян 

---

In [16]:
def print_top_words(model, feature_names, n_top_words=7):
    for topic_idx, topic in enumerate(model.components_):
        message = "Тема %d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

---

In [17]:
#df['text_clean']

In [18]:
# tf features 
tf_vectorizer = CountVectorizer( max_df=0.95, min_df=2, max_features=n_features )
tf = tf_vectorizer.fit_transform(df['text_clean'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [19]:
# LDA - латентное размещение Дирихле
lda = LatentDirichletAllocation( n_components=n_components, max_iter=5, 
                                learning_method='online', learning_offset=50.,
                                random_state=0 ).fit(tf)
print('\nLDA:\n')
print_top_words(lda, tf_feature_names)


LDA:

Тема 0: это очень которые время просто лет нужно
Тема 1: президент сша трамп президента заявил декабря глава
Тема 2: рублей автомобиль модели автомобилей модель автомобиля авто
Тема 3: беларуси tut декабря установлена внимание версия браузер
Тема 4: народов коренных севера малочисленных коми проекта края
Тема 5: object савченко нефти нефть тгк индекс оао
Тема 6: года году это также компании декабря которые
Тема 7: декабря tut время дтп фото результате области
Тема 8: года также декабря россии тысяч сообщает данным
Тема 9: динамо лучший мяч мира место стал чемпионата


---

In [20]:
# tf-idf features 
tfidf_vectorizer = TfidfVectorizer( max_df=0.95, min_df=2, max_features=n_features)

tfidf = tfidf_vectorizer.fit_transform(df['text_clean'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [21]:
# NMF (Frobenius norm) - неотрицательное матричное разложение
nmf = NMF( n_components=n_components, random_state=1,alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(Frobenius norm):\n')
print_top_words( nmf, tfidf_feature_names )


NMF(Frobenius norm):

Тема 0: это года которые году также декабря время
Тема 1: трамп сша трампа дональд президент избранный президента
Тема 2: дтп водитель результате области мвд декабря происшествия
Тема 3: flash adobe javascript player проигрывателя html5 браузер
Тема 4: савченко украины партии надежда заявила лидер действия
Тема 5: рублей млн млрд долларов тысяч года году
Тема 6: динамо чемпионата матче мира очков матча лиги
Тема 7: народов севера коренных малочисленных края фестиваль июля
Тема 8: россии путин президент президента заявил глава россия
Тема 9: алеппо города сирии жителей восточной сутки тысяч


---

In [22]:
# NMF (generalized Kullback-Leibler divergence)  
nmf = NMF( n_components=n_components, random_state=1, beta_loss='kullback-leibler', 
          solver='mu', max_iter=1000, alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(generalized Kullback-Leibler divergence):\n')
print_top_words(nmf, tfidf_feature_names )


NMF(generalized Kullback-Leibler divergence):

Тема 0: декабря года это которая также году фото
Тема 1: сша президент года ранее президента главы дональд
Тема 2: декабря области время результате tut пресс сообщает
Тема 3: tut декабря adobe версия видео браузер внимание
Тема 4: сегодня украины вопрос это декабря словам сообщил
Тема 5: млн будут года области рублей рамках народов
Тема 6: место мира декабря который стал матче года
Тема 7: ученые жизни которые лет людей люди могут
Тема 8: россии декабря заявил глава государства президент россия
Тема 9: года также компания который сообщает ранее компании
